In [1]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from sae import SaeConfig, SaeLayerRangeTrainer, TrainLayerRangeConfig
from sae.data import chunk_and_tokenize

%load_ext autoreload
%autoreload 2

In [2]:
MODEL = "EleutherAI/pythia-160m"
dataset = load_dataset(
    "togethercomputer/RedPajama-Data-1T-Sample",
    split="train",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenized = chunk_and_tokenize(dataset, tokenizer)


gpt = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map={"": "cuda"},
    torch_dtype=torch.bfloat16,
)

cfg = TrainLayerRangeConfig(SaeConfig(gpt.config.hidden_size), batch_size=16)
trainer = SaeLayerRangeTrainer(cfg, tokenized, gpt)

trainer.fit()

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Map (num_proc=48):   0%|          | 0/930514 [00:00<?, ? examples/s]